Оценка RAG

In [20]:
from typing import List
import os
import asyncio
import json
import numpy as np
from react_agent.tools import search 
from react_agent import graph
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas import EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, ResponseRelevancy



def retrieve(query: str, top_k: int = 10) -> List[str]:
    """
    Вызов Qdrant-инструмента search() для получения до top_k фрагментов контекста.
    """
    raw = search(query)
    # search() возвращает строки, разделённые сериями ===== и заголовками источников
    # Разбиваем по заголовку каждого результата
    parts = [part.strip() for part in raw.split("==========Актуально в период:") if part.strip()]
    # Берём первые top_k результатов
    return parts[:top_k]


In [21]:
print(graph.input_schema)


<class 'langchain_core.utils.pydantic.ReAct AgentInput'>


Создание датасета

In [33]:
import asyncio
import json
from langchain_core.messages import HumanMessage

async def main():
    # Читаем вопросы из файла
    questions = open('../data/testing_data/test_cases.txt', encoding='utf-8').read().splitlines()
    dataset = []

    for q in questions:
        try:
            # Получаем релевантные контексты для вопроса
            contexts = retrieve(q)  # предполагаемая функция retrieve

            # Формируем prompt для агента
            prompt = f"Вопрос пользователя:\n{q}"

            # Асинхронно вызываем граф
            response_obj = await graph.ainvoke({"messages": [HumanMessage(content=q)]})

            # Извлекаем текст из ответа агента
            response_text = response_obj['messages'][-1].content

            # Добавляем запись в датасет
            dataset.append({
                "user_input": q,
                "retrieved_contexts": contexts,
                "response": response_text
            })

            print(f"{q} → {response_text}")
        except Exception as e:
            print(f"Ошибка при обработке вопроса '{q}': {e}")


        # Сохраняем собранный датасет в JSON-файл
    with open('dataset.json', 'w', encoding='utf-8') as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4)
        

if __name__ == "__main__":
    asyncio.run(main())


Request error: 404 Client Error: Not Found for url: https://llm.api.cloud.yandex.net/foundationModels/v1/completion
Здравствуйте, а подскажите флюорография и справка с анализами от дерматолога сколько действительны, т.е, за сколько до заселения должны быть сделаны? → Согласно доступной информации, для заселения в общежитие МАИ требуется флюорографическое исследование сроком не позднее одного года. О сроках действия справки от дерматолога точной информации нет, однако рекомендуется проходить обследование ежегодно.

Для уточнения актуальной информации и возможных изменений в требованиях, рекомендую связаться с дирекцией студгородка МАИ по телефону +7 499 195-95-14.
Request error: 404 Client Error: Not Found for url: https://llm.api.cloud.yandex.net/foundationModels/v1/completion
общежития и физкультуры нужна справка 086у. Достаточно будет отдать оригинал например а общежитие, в другое место копию? → Согласно найденной информации, для заселения в общежитие МАИ требуется предоставление либ

KeyboardInterrupt: 

In [ ]:
from ragas import EvaluationDataset
eval_ds = EvaluationDataset.from_list(dataset)


Оценка

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, ResponseRelevancy, AnswerCorrectness, AnswerCompleteness, ContextPrecision, ContextRecall
llm = ChatOpenAI(
        model="gpt://" + os.environ["FOLDER_ID"] + '/yandexgpt/rc',
        api_key=os.environ["API_KEY"],
        temperature=0.1,
        base_url="https://llm.api.cloud.yandex.net/v1"
    )
evaluator = LangchainLLMWrapper(llm)
metrics = [
    LLMContextRecall(),
    Faithfulness(),
    ResponseRelevancy(),
    AnswerCorrectness(),
    AnswerCompleteness(),
    ContextPrecision(),
    ContextRecall()
]
results = evaluate(
    dataset=eval_ds,
    metrics=metrics,
    llm=evaluator
)
print(results)
